# Tutoria11: Llama Factory/Deepspeed 对于 Qwen2.5-32B-Instruct多机多卡的全参数sft微调过程

本节展示如何在 SCOW 平台上使用多机多卡对于Qwen2.5-32B-Instruct的全参数微调。
Llama Factory这个架构会默认以torchrun启动训练，进行多机多卡训练时，torchrun会尝试通过创建一个端口配置主机服务器，从而让从机进行连接训练，然而SCOW平台上开启新端口需要向管理员申请。因此，使用Llama Factory内置的Deepspeed(也可以直接使用Deepspeed)通过ssh端口(22端口)直接进行分布式训练，可以达到不需要创建主机服务器也可以在SCOW平台上进行多机多卡分布式训练。同时，使用集群的IB高速网络，可以大大加快训练速度。
本文将详细介绍如何使用Deepspeed在SCOW平台上进行多机多卡分布式训练，并且罗列出可能会遇到问题和解决方法。

分以下几步来实现：
1. 环境安装、应用创建以及下载模型
2. ssh免密配置
3. 安装Llama Factory
4. 配置hostfile
5. 训练数据处理
6. 环境配置+启动训练
7. 常见问题

Qwen 系列模型是由阿里巴巴开发的。Qwen 模型系列包括不同规模的模型，参数范围从 0.5 到 720 亿，适用于各种应用场景，如文本生成、翻译、问答等。

## 1. 环境安装、应用创建以及下载模型

首先在联网的命令行中创建conda环境:
```bash
conda create -n tutorial11 python=3.9
conda activate tutorial11
conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia
pip install --upgrade notebook jupyterlab transformers huggingface_hub accelerate
```

（ pytorch 版本需与 cuda 版本对应，请查看版本对应网站：https://pytorch.org/get-started/previous-versions ，通过 nvidia-smi 命令可查看 cuda 版本）

然后创建JupyterLab应用, `Conda环境名`请填写`tutorial11`, 建议的硬件资源为两个节点，每个节点4张A800显卡。创建应用后, 进入应用并打开本文件。

CUDA Version: 12.1; Torch Version: 2.3.1

创建环境后，通过镜像源下载pdsh软件包并安装，注意将<虚拟环境路径>替换成实际路径：
```bash
wget https://mirrors.tuna.tsinghua.edu.cn/ubuntu/pool/universe/p/pdsh/
pdsh_2.34.orig.tar.gz
tar -xzvf pdsh_2.34.orig.tar.gz
cd pdsh-2.34/
./configure --prefix=<虚拟环境路径> --with-ssh
make -j 4 && make install
```
激活虚拟环境后，可以通过以下指令进行验证安装：
```bash
pdsh -V
```
然后通过以下指令下载模型，注意将<path>替换成实际下载目录：
```bash
pip install modelscope
modelscope download --model Qwen/Qwen2.5-32B-Instruct --local_dir <path>
```

## 2. ssh免密配置

流程以假设申请的两个节点l12gpu28和l12gpu29为例子，创建两个terminal，分别使用ssh登录28和29节点，注意将username替换成实际username，需要输入密码：

```bash
ssh username@l12gpu28
ssh username@l12gpu29
```
在其中一个节点上生成sshkey，然后复制到另外一个节点上，注意第一条指令执行后需要多次按回车：
```bash
ssh-keygen -t rsa -b 4096
ssh-copy-id -i ~/.ssh/id_rsa.pub username@l12gpu29
```
SCOW平台默认共享硬盘，所以一般只需要从一个节点复制到另外一个节点即可，但为了确保无意外发生，在l12gpu29节点上执行一样的步骤，确保两个节点都可以免密登录到另外一个节点，若提示密钥已存在，可以忽略此步骤。

分别在l12gpu28和l12gpu29节点上测试，如不需要密码则配置成功：
```bash
ssh username@l12gpu29
ssh username@l12gpu28
```

## 3. 安装Llama Factory

在联网的terminal中下载并安装llama factory，注意下载安装之前需要先激活虚拟环境:
```bash
git clone –depth 1 http://github.com/hiyouga/LLaMA_Factory.git
Cd LLaMA-Factory
pip install –e ".[torch,metrics]"
```



## 4. 配置hostfile

在llama factory目录下创建一个hostfile（无需后缀），输入以下信息，注意记得把l12gpu28和l12gpu29替换成实际节点：
```bash
l12gpu28 slots=4
l12gpu29 slots=4
```

## 5.训练数据处理
Llama Factory目录中data/dataset_info.json文件包含了所有经过预处理的本地数据集以及在线数据集。如果希望使用自定义数据集，请 务必 在 dataset_info.json 文件中添加对数据集及其内容的定义。Llama Factory目前支持Alpaca格式和ShareGPT格式的数据集，以下是Alpace格式的一个例子：
```bash
"example.json"
{
  "instruction": "请帮我解决这个数学问题。",
  "input": "3+6等于几。",
  "output": "3+6=9，所以答案是9。"
},
```
在进行指令监督微调时， instruction 列对应的内容会与 input 列对应的内容拼接后作为最终的人类输入，即人类输入为 instruction\ninput。而 output 列对应的内容为模型回答。 在上面的例子中，用户最终的输入是：
“请帮我解决这个数学问题。3+6等于几。”
模型的回答是：
“3+6=9，所以答案是9。
更详细的sft微调可以通过以下链接学习：

https://llamafactory.readthedocs.io/zh-cn/latest/getting_started/data_preparation.html
https://zhuanlan.zhihu.com/p/695287607

## 6.环境配置+启动训练
通过指令进行环境配置，注意按照实际情况修改：
```bash
#!/bin/bash
set –x
#禁用HF的tokenizers库，避免多线程冲突，提升稳定性，会导致分词速度略微下降
export TOKENIZERS_PARALLELISM=false
#用特定的NCCL版本确保兼容性
export NCCL_VERSION=2.17.1
#指定NCLL接口类型
export NCCL_IB_HCA=mlx5
#设置IB GID index未为3，优化多节点通信
export NCCL_IB_GID_INDEX=3
#设置超时时间，避免误判
export NCCL_IB_TIMEOUT=22
#设置流量类别（可选）
export NCCL_IB_TC=136
#设置IB服务级别（可选）
export NCCL_IB_SL=5
#允许在多个队列上拆分数据（可选，提高宽带利用率）
export NCCL_IB_SPLIT_DATA_ON_QPS=1
#设置IB连接队列数量为8（按照显卡数量选择）
export NCCL_IB_QPS_PER_CONNECTION=8
#设置IB操作失败后的重试次数
export NCCL_IB_RETRY_CNT=15
#禁用NCCL的网络插件，强制使用IB或者以太网通信
export NCCL_NET_PLUGIN=none
#指定网络接口为eno1，可以用ifconfig指令来查看接口
export NCCL_SOCKET_IFNAME=eno1
#减日志输出量，需要更相信的错误信息可选择INFO
export NCCL_DEBUG=WARN
#设置每个cuda设备最大的并发连接数为1
export CUDA_DEVICE_MAX_CONNECTIONS=1
```
启动训练，注意按照实际情况修改，NNODES是节点个数，base_model_path是模型下载路径，model_output是训练模型保存的路径，HOSTFILE是配置的hostfile文件路径，同时修改Llama Factory目录内的/examples/deepspeed/ds_z3_config.json的Deepspeed配置，以下例子用了stage3：
```bash
DIR=`pwd`
nvidia-smi
GPUS_PER_NODE=$(python3 -c 'import torch; print(torch.cuda.device_count())')
NNODES=2
NODE_RANK=${RANK:-0}
WORLD_SIZE=$(($GPUS_PER_NODE * $NNODES))
if [ "$MASTER_ADDR" == "localhost" ]; then MASTER_ADDR=$(hostname); fi

export MLP_WORKER_0_HOST=$KUBERNETES_POD_IP
export MLP_WORKER_0_PORT=$MASTER_PORT

base_model_path="<model_path>"
model_output=<out_dir>
HOSTFILE=<path_to_hostfile>

deepspeed --hostfile $HOSTFILE --ssh_port 22 /src/train.py \
    --deepspeed /examples/deepspeed/ds_z3_config.json \
        --stage sft \
        --do_train \
        --model_name_or_path $base_model_path \
        --dataset <dataset_name> \
        --template qwen \
        --finetuning_type full \
        --output_dir <output_path> \
        --overwrite_cache \
        --per_device_train_batch_size 1 \
        --gradient_accumulation_steps 1 \
        --lr_scheduler_type cosine \
        --logging_steps 10 \
        --save_steps 10000000 \
        --learning_rate 1e-5 \
        --num_train_epochs 5.0 \
        --bf16 True \
        --plot_loss \
        --flash_attn fa2 \
        --overwrite_output_dir
```

## 7.常见错误
7.1 wandb连接错误
    分布式训练不会询问用户是否要开启wandb，所以要主动禁用wandb，避免报错，在Llama Factory目录下，进入src/train.py，在开头加入：

```bash
Import os
os.environ["WANDB_DISABLED"] = "true"
```

7.2 卡在/.cache/torch_extensions

    这里需要清理掉.cache/torch_extensions，只需要把这个文件夹里的东西删除即可。

7.3 RuntimeError: Ninja is required to load C++ extensions
    如果已经安装了Ninja但仍然出现此错误，需要找到Ninja的安装路径
```bash
local_env = os.environ.copy()
local_env["PATH"]=<conda环境地址>
os.environ.update(local_env)
```

7.4 出现transport/net_ib.cc:101 NCCL WARN NET/IB : mlx5_0:1 Got async event : local access violation work queue error

    出现这个错误，一般是使用的两个节点并不相邻，在这里可以强制禁止使用IB高速网络，使用传统的TCP进行分布式训练，但是性能会有所降低，在环境配置中加入：
```bash
export NCCL_IB_DISABLE=1
```
    也可以取消作业，重新申请节点，比如原来是l12gpu06和l12gpu23，可以取消掉并重新申请，SCOW会尽量分配两个相邻的节点，比如l12gpu28和l12gpu29

---

> 作者: 黎颖; 龙汀汀
>
> 联系方式: yingliclaire@pku.edu.cn;   l.tingting@pku.edu.cn